In [47]:
import socket
import time

HOST = "192.168.1.20"
PORT = 10003


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    
    message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{120.420})(3,0)"
    #message = n + m  # Concatenate n and m
    time.sleep(0.001)
    s.sendall(message.encode())  # Send the concatenated message
    


In [13]:
#jnr run without  initial index finger and need above first
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket
import time

HOST = "192.168.1.20"
PORT = 10003

# Socket setup
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

f = 134.420

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width of the webcam frame
cap.set(4, 720)   # Set height of the webcam frame

# Hand Detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Calibration for distance conversion
pixels_per_cm = 37  # This value should be calibrated according to your setup

# Variables to store initial position
initial_position = None
current_position = None

def send_message_and_confirm(sock, message):
    try:
        print(message)
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()  # Assuming the server sends a response
        return response == "True"  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending message: {e}")
        return False

try:
    # Main loop
    while True:
        success, img = cap.read()
        img = cv2.flip(img, 1)  # Flip the image horizontally

        hands, img = detector.findHands(img, draw=True)  # Draw hand landmarks

        if hands:
            hand = hands[0]
            lmList = hand["lmList"]  # List of landmarks
            bbox = hand["bbox"]  # Bounding box info x,y,w,h

            index_tip_x, index_tip_y = lmList[8][:2]  # Index finger tip coordinates

            if initial_position is None:
                initial_position = index_tip_x  # Set the initial position

            current_position = index_tip_x
            dx = current_position - initial_position  # Calculate the difference in x-coordinates
            distance_cm = dx / pixels_per_cm  # Convert pixels to centimeters
            # Display the distance moved
            cvzone.putTextRect(img, f'Moved: {distance_cm:.2f} cm', (50, 50))

            updated = False
            if distance_cm > 0 and  (distance_cm % 3 ==0):
                new_f = min(f + 10, 175)  # Add 10 but do not exceed 175
                if new_f != f:
                    message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{new_f:.3f})(3,0)"
                    if send_message_and_confirm(sock, message):
                        f = new_f
                        updated = True
            elif distance_cm < 0 and (distance_cm % 3 ==0):
                new_f = max(f - 20, 10)   # Subtract 20 but do not go below 10
                if new_f != f:
                    message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{new_f:.3f})(3,0)"
                    if send_message_and_confirm(sock, message):
                        f = new_f
                        updated = True

            if updated:
                print(f'Updated coordinate: {f}')

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
    sock.close()


In [13]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket
import time

HOST = "192.168.1.20"
PORT = 10003

# Socket setup
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

f = 134.420

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width of the webcam frame
cap.set(4, 720)   # Set height of the webcam frame

# Hand Detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Calibration for distance conversion
pixels_per_cm = 37  # This value should be calibrated according to your setup

# Variables to store initial position
initial_position = None
current_position = None

# Flag to control the flow of execution
initial_message_sent = False

'''def send_initial_message():
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            s.connect((HOST, PORT))
            m = "1,(17.740,0.700,606.050,133.670,87.030,134.420)(3,0)"
            print(f"Sending initial message: {m}")
            time.sleep(0.001)
            s.sendall(m.encode())
            response = s.recv(1024).decode()  # Assuming the server sends a response
            print(f"Initial message response: {response}")
            return response == "True"  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending initial message: {e}")
        return False
'''
def send_message_and_confirm(sock, message):
    try:
        print(f"Sending message: {message}")
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()  # Assuming the server sends a response
        print(f"Message response: {response}")
        return response == "True"  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending message: {e}")
        return False

try:
    # Main loop
    while True:
        success, img = cap.read()
        img = cv2.flip(img, 1)  # Flip the image horizontally

        hands, img = detector.findHands(img, draw=True)  # Draw hand landmarks

        if hands:
            hand = hands[0]
            lmList = hand["lmList"]  # List of landmarks
            bbox = hand["bbox"]  # Bounding box info x,y,w,h

            index_tip_x, index_tip_y = lmList[8][:2]  # Index finger tip coordinates

            fingers = detector.fingersUp(hand)
            if fingers.count(1) == 1 and not initial_message_sent:
                # Send the initial message when one finger is detected
                print("One finger detected, sending initial message.")
                if not initial_message_sent:
                    initial_message_sent = True
                    try:
                        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                            s.connect((HOST, PORT))
                            m = "1,(17.740,0.700,606.050,133.670,87.030,134.420)(3,0)"
                            print(f"Sending initial message: {m}")
                            time.sleep(0.001)
                            s.sendall(m.encode())
                            response = s.recv(1024).decode() 
                    except Exception as e:
                        print(f"Error sending initial message: {e}")
                print("Initial message sent and confirmed.")
            else:
                print("Failed to send initial message. Retrying...")
                continue

            if initial_message_sent:
                if initial_position is None:
                    initial_position = index_tip_x  # Set the initial position

                current_position = index_tip_x
                dx = current_position - initial_position  # Calculate the difference in x-coordinates
                distance_cm = dx / pixels_per_cm  # Convert pixels to centimeters
                # Display the distance moved
                cvzone.putTextRect(img, f'Moved: {distance_cm:.2f} cm', (50, 50))

                updated = False
                if distance_cm > 0 and (distance_cm % 3 == 0):
                    new_f = min(f + 10, 175)  # Add 10 but do not exceed 175
                    if new_f != f:
                        message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{new_f:.3f})(3,0)"
                        if send_message_and_confirm(sock, message):
                            f = new_f
                            updated = True
                elif distance_cm < 0 and (abs(distance_cm) % 3 == 0):
                    new_f = max(f - 20, 10)   # Subtract 20 but do not go below 10
                    if new_f != f:
                        message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{new_f:.3f})(3,0)"
                        if send_message_and_confirm(sock, message):
                            f = new_f
                            updated = True

                if updated:
                    print(f'Updated coordinate: {f}')

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
    sock.close()


Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. Retrying...
Failed to send initial message. 

In [14]:
#run successful
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket
import time

HOST = "192.168.1.20"
PORT = 10003

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

f_x = 134.420  
f_y = 606.050 


cap = cv2.VideoCapture(0)
cap.set(3, 1280)  
cap.set(4, 720)   

detector = HandDetector(detectionCon=0.8, maxHands=1)

pixels_per_cm = 37  


initial_position_x = None
initial_position_y = None

initial_message_sent = False

def send_initial_message(sock):
    try:
        m = "1,(+17.740,+0.700,+676.050,+133.670,+87.030,+134.420)(3,0)"
        time.sleep(0.1)
        sock.sendall(m.encode())
        response = sock.recv(1024).decode()  
        return True 
    except Exception as e:
        print(f"Error sending initial message: {e}")
        return False

def send_message_and_confirm(sock, message):
    try:
        print(f"Sending message: {message}")
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()
        print(f"Received response: {response}")
        return response == True 
    except Exception as e:
        print(f"Error sending message: {e}")
        return False

try:
    flag= False
    while True:
        success, img = cap.read()
        img = cv2.flip(img, 1)  
        hands, img = detector.findHands(img, draw=True)  

        if hands:
            hand = hands[0]
            lmList = hand["lmList"] 
            bbox = hand["bbox"] 

            index_tip_x, index_tip_y = lmList[8][:2] 

            fingers = detector.fingersUp(hand)
            if fingers.count(1) == 1 and not initial_message_sent:
               
                if send_initial_message(sock):
                    initial_message_sent = True
                    print("Initial message sent and confirmed.")
                else:
                    print("Failed to send initial message. Retrying...")
                continue

            if initial_message_sent:
                if initial_position_x is None:
                    initial_position_x = index_tip_x  
                if initial_position_y is None:
                    initial_position_y = index_tip_y  

                current_position_x = index_tip_x
                dx = current_position_x - initial_position_x
                distance_cm_x = dx / pixels_per_cm

                current_position_y = index_tip_y
                dy = current_position_y - initial_position_y  
                distance_cm_y = dy / pixels_per_cm 

                
                cvzone.putTextRect(img, f'Moved X: {distance_cm_x:.2f} cm', (50, 50))
                cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 100))

                updated = False
                if distance_cm_x > 0 and int(distance_cm_x) % 3 == 0:
                    new_f_x = min(f_x + 10, 175)
                    if new_f_x != f_x:
                        message = f"1,(+17.740,+0.700,+606.050,+133.670,+{f_y:.3f},+{new_f_x:.3f})(3,0)"
                        if send_message_and_confirm(sock, message):
                            f_x = new_f_x
                            updated = True
                elif distance_cm_x < 0 and int(abs(distance_cm_x)) % 3 == 0:
                    new_f_x = min(f_x - 20, 10)
                    if new_f_x != f_x:
                        message = f"1,(+17.740,+0.700,+606.050,+133.670,+{f_y:.3f},+{new_f_x:.3f})(3,0)"
                        if send_message_and_confirm(sock, message):
                            f_x = new_f_x
                            updated = True

                if distance_cm_y > 0:
                    new_f_y = max(f_y - 20, 505)  
                    if new_f_y != f_y:
                        message = f"1,(+17.740,+0.700,+{new_f_y:.3f},+133.670,+87.030,+{f_x:.3f})(3,0)"
                        if send_message_and_confirm(sock, message):
                            f_y = new_f_y
                            updated = True
                elif distance_cm_y < 0:
                    new_f_y = min(f_y + 20, 850)  
                    if new_f_y != f_y:
                        message = f"1,(+17.740,+0.700,+{new_f_y:.3f},+133.670,+87.030,+{f_x:.3f})(3,0)"
                        if send_message_and_confirm(sock, message):
                            f_y = new_f_y
                            updated = True

                if fingers.count(1) == 0:
                    message = f"1,(+17.740,+0.700,+481.840,+133.670,+87.030,+134.420)(3,0)"
                    if send_message_and_confirm(sock, message):
                        updated = True
                        flag=True

                if updated:
                    print(f'Updated coordinates: f_x={f_x}, f_y={f_y}')
            if flag == True:
                break

        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
    sock.close()


Initial message sent and confirmed.
Sending message: 1,(+17.740,+0.700,+586.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+606.050,+133.670,+606.050,+10.000)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+586.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+626.050,+133.670,+87.030,+13

In [3]:
#run successful(world axis)
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket
import time

HOST = "192.168.1.20"
PORT = 10003

# Socket setup
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

f_x = 134.420  # Initial value for horizontal movement
f_y = 606.050 # Initial value for vertical movement

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width of the webcam frame
cap.set(4, 720)   # Set height of the webcam frame

# Hand Detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Calibration for distance conversion
pixels_per_cm = 37  # This value should be calibrated according to your setup

# Variables to store initial positions
initial_position_x = None
initial_position_y = None

# Flag to control the flow of execution
initial_message_sent = False

def send_initial_message(sock):
    try:
        m = "1,(+17.740,+0.700,+676.050,+133.670,+87.030,+134.420)(3,0)"
        time.sleep(0.1)
        sock.sendall(m.encode())
        response = sock.recv(1024).decode()  # Assuming the server sends a response
        return True  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending initial message: {e}")
        return False

def send_message_and_confirm(sock, message):
    try:
        print(f"Sending message: {message}")
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()  # Assuming the server sends a response
        print(f"Received response: {response}")
        return  True  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending message: {e}")
        return False
try:
    # Main loop
    while True:
        success, img = cap.read()
        img = cv2.flip(img, 1)  # Flip the image horizontally

        hands, img = detector.findHands(img, draw=True)  # Draw hand landmarks
        
        updated = False  # Initialize updated variable here
        check = False
        
        if hands:
            hand = hands[0]
            lmList = hand["lmList"]  # List of landmarks
            bbox = hand["bbox"]  # Bounding box info x,y,w,h

            index_tip_x, index_tip_y = lmList[8][:2]  # Index finger tip coordinates

            fingers = detector.fingersUp(hand)
            if fingers.count(1) == 1 and not initial_message_sent:
                # Send the initial message when one finger is detected
                if send_initial_message(sock):
                    initial_message_sent = True
                    print("Initial message sent and confirmed.")
                else:
                    print("Failed to send initial message. Retrying...")
                continue

            if initial_message_sent:
                if initial_position_x is None:
                    initial_position_x = index_tip_x  # Set the initial position for horizontal movement
                if initial_position_y is None:
                    initial_position_y = index_tip_y  # Set the initial position for vertical movement

                # Calculate the distance moved and adjust the position
                current_position_x = index_tip_x
                dx = current_position_x - initial_position_x  # Calculate the difference in x-coordinates
                distance_cm_x = dx / pixels_per_cm  # Convert pixels to centimeters for horizontal movement

                current_position_y = index_tip_y
                dy = current_position_y - initial_position_y  # Calculate the difference in y-coordinates
                distance_cm_y = dy / pixels_per_cm  # Convert pixels to centimeters for vertical movement

                cvzone.putTextRect(img, f'Moved X: {distance_cm_x:.2f} cm', (50, 50))
                cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 100))
                flag = False

                # Determine if the movement is primarily horizontal or vertical
                is_horizontal_movement = abs(dx) > abs(dy)
                is_vertical_movement = abs(dy) > abs(dx)

                # Horizontal movement (x)
                if is_horizontal_movement:
                    if distance_cm_x > 0:
                        # Add 20 units if positive distance is divisible by 2
                        if int(distance_cm_x) % 2 == 0:
                            new_f_x = min(f_x + 20, 175)
                            if new_f_x != f_x:
                                message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{new_f_x:.3f})(3,0)"
                                if send_message_and_confirm(sock, message):
                                    f_x = new_f_x
                                    updated = True
                    elif distance_cm_x < 0:
                        # Subtract 15 units if negative distance is divisible by 2
                        if int(abs(distance_cm_x)) % 2 == 0:
                            new_f_x = max(f_x - 15, 10)
                            if new_f_x != f_x:
                                message = f"1,(+17.740,+0.700,+606.050,+133.670,+87.030,+{new_f_x:.3f})(3,0)"
                                if send_message_and_confirm(sock, message):
                                    f_x = new_f_x
                                    updated = True

                # Vertical movement (y)
                if is_vertical_movement:
                    if distance_cm_y > 0:
                        # Subtract 20 units if positive distance is divisible by 2
                        if int(distance_cm_y) % 2 == 0:
                            new_f_y = max(f_y - 20, 505)
                            if new_f_y != f_y:
                                message = f"1,(+17.740,+0.700,+{new_f_y:.3f},+133.670,+87.030,+134.420)(3,0)"
                                if send_message_and_confirm(sock, message):
                                    f_y = new_f_y
                                    updated = True
                    elif distance_cm_y < 0:
                        # Add 15 units if negative distance is divisible by 2
                        if int(abs(distance_cm_y)) % 2 == 0:
                            new_f_y = min(f_y + 15, 850)
                            if new_f_y != f_y:
                                message = f"1,(+17.740,+0.700,+{new_f_y:.3f},+133.670,+87.030,+134.420)(3,0)"
                                if send_message_and_confirm(sock, message):
                                    f_y = new_f_y
                                    updated = True

                    if fingers.count(1) == 0:
                        message = f"1,(+17.740,+0.700,+481.840,+133.670,+87.030,+134.420)(3,0)"
                        if send_message_and_confirm(sock, message):
                            updated = True
                            flag=True

                    if updated:
                        print(f'Updated coordinates: f_x={f_x}, f_y={f_y}')
                if flag == True:
                    check = True
                    break
            if check:
                break
                    
        cv2.imshow("Image", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    cap.release()
    cv2.destroyAllWindows()
    sock.close()


Initial message sent and confirmed.
Sending message: 1,(+17.740,+0.700,+606.050,+133.670,+87.030,+154.420)(3,0)
Received response: True
Sending message: 1,(+17.740,+0.700,+621.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Updated coordinates: f_x=154.42, f_y=621.05
Sending message: 1,(+17.740,+0.700,+636.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Updated coordinates: f_x=154.42, f_y=636.05
Sending message: 1,(+17.740,+0.700,+651.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Updated coordinates: f_x=154.42, f_y=651.05
Sending message: 1,(+17.740,+0.700,+666.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Updated coordinates: f_x=154.42, f_y=666.05
Sending message: 1,(+17.740,+0.700,+681.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Updated coordinates: f_x=154.42, f_y=681.05
Sending message: 1,(+17.740,+0.700,+696.050,+133.670,+87.030,+134.420)(3,0)
Received response: True
Updated coordinates: f_x=154.42, f_y=696.05


In [3]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket
import time

HOST = "192.168.1.20"
PORT = 10003

# Socket setup
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

# Initial joint positions
J1 = 0.400 
J2 = -113.980
J3 = 162.100
J4 = 0.560
J5 = 43.930
J6 = 1.730

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width of the webcam frame
cap.set(4, 720)   # Set height of the webcam frame

# Hand Detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Calibration for distance conversion
pixels_per_cm = 37  

# Variables to store initial positions
initial_position_x = None
initial_position_y = None

def send_message_and_confirm(sock, message):
    try:
        print(f"Sending message: {message}")
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()  # Assuming the server sends a response
        print(f"Received response: {response}")
        return response.strip() == "True"  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending message: {e}")
        return False

# Main loop
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flip the image horizontally

    hands, img = detector.findHands(img, draw=True)  
    
    updated = False  # Initialize updated variable here
    
    if hands:
        hand = hands[0]
        lmList = hand["lmList"]  # List of landmarks
        bbox = hand["bbox"]  # Bounding box info x,y,w,h

        fingers = detector.fingersUp(hand)

        if initial_position_x is None:
            initial_position_x = lmList[8][0]  # Set the initial position for x movement
        if initial_position_y is None:
            initial_position_y = lmList[8][1]  # Set the initial position for y movement

        current_position_x = lmList[8][0]
        dx = current_position_x - initial_position_x  # Calculate the difference in x-coordinates
        distance_cm_x = dx / pixels_per_cm  # Convert pixels to centimeters for x movement

        current_position_y = lmList[8][1]
        dy = current_position_y - initial_position_y  # Calculate the difference in y-coordinates
        distance_cm_y = dy / pixels_per_cm  # Convert pixels to centimeters for y movement

        cvzone.putTextRect(img, f'Moved X: {distance_cm_x:.2f} cm', (50, 50))
        cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 100))

        joint_value = None

        # Determine which joint to control based on finger count
        if fingers == [1, 0, 0, 0, 0]:  # One finger up controls J1
            joint_value = J1
            movement = distance_cm_x
            cvzone.putTextRect(img, "J1 activated", (50, 150))  # Display joint activation
        elif fingers == [0, 1, 0, 0, 0]:  # Two fingers up controls J2
            joint_value = J2
            movement = distance_cm_y
            cvzone.putTextRect(img, "J2 activated", (50, 150))  # Display joint activation
        elif fingers == [0, 0, 1, 0, 0]:  # Three fingers up controls J3
            joint_value = J3
            movement = -distance_cm_y  # Invert movement for J3
            cvzone.putTextRect(img, "J3 activated", (50, 150))  # Display joint activation
        elif fingers == [0, 0, 0, 1, 0]:  # Four fingers up controls J4
            joint_value = J4
            movement = distance_cm_x
            cvzone.putTextRect(img, "J4 activated", (50, 150))  # Display joint activation
        elif fingers == [0, 0, 0, 0, 1]:  # Five fingers up controls J5
            joint_value = J5
            movement = -distance_cm_y  # Invert movement for J5
            cvzone.putTextRect(img, "J5 activated", (50, 150))  # Display joint activation
        '''elif fingers == [0, 0, 0, 0, 1]:  # Thumb up controls J6
            joint_value = J6
            movement = distance_cm_x
            cvzone.putTextRect(img, "J6 activated", (50, 150))  # Display joint activation'''

        if joint_value is not None:
            # Determine if the movement is positive or negative
            if movement > 0:
                adjustment = 5 if int(movement) % 2 == 0 else 0
            else:
                adjustment = -5 if int(abs(movement)) % 2 == 0 else 0

            joint_value += adjustment

            # Update the correct joint value
            if fingers == [1, 0, 0, 0, 0]:  # One finger up controls J1
                J1 = joint_value
            '''elif fingers == [0, 1, 0, 0, 0]:  # Two fingers up controls J2
                J2 = joint_value
            elif fingers == [0, 0, 1, 0, 0]:  # Three fingers up controls J3
                J3 = joint_value
            elif fingers == [0, 0, 0, 1, 0]:  # Four fingers up controls J4
                J4 = joint_value
            elif fingers == [0, 0, 0, 0, 1]:  # Five fingers up controls J5
                J5 = joint_value
            elif fingers == [0, 0, 0, 0, 1]:  # Thumb up controls J6
                J6 = joint_value'''

            # Construct the message for the updated joint positions
            message = f"1,({J1:.3f},{J2:.3f},{J3:.3f},{J4:.3f},{J5:.3f},{J6:.3f})(3,0)"
            if send_message_and_confirm(sock, message):
                updated = True
                print(f'Updated joints: J1={J1}, J2={J2}, J3={J3}, J4={J4}, J5={J5}, J6={J6}')

        if updated:
            initial_position_x = current_position_x
            initial_position_y = current_position_y

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
sock.close()


Sending message: 1,(0.400,-113.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Updated joints: J1=0.4, J2=-113.98, J3=162.1, J4=0.56, J5=43.93, J6=1.73
Sending message: 1,(-4.600,-113.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Updated joints: J1=-4.6, J2=-113.98, J3=162.1, J4=0.56, J5=43.93, J6=1.73
Sending message: 1,(0.400,-113.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Updated joints: J1=0.40000000000000036, J2=-113.98, J3=162.1, J4=0.56, J5=43.93, J6=1.73
Sending message: 1,(5.400,-113.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Updated joints: J1=5.4, J2=-113.98, J3=162.1, J4=0.56, J5=43.93, J6=1.73
Sending message: 1,(0.400,-113.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Updated joints: J1=0.40000000000000036, J2=-113.98, J3=162.1, J4=0.56, J5=43.93, J6=1.73
Sending message: 1,(5.400,-113.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Updated joints: J1=5.4, J2=-113.98, J3=162.1, J4=

In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket

HOST = "192.168.1.20"
PORT = 10003

# Socket setup
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    sock.connect((HOST, PORT))
except ConnectionRefusedError:
    print("Error: Connection refused. Please check if the server is running.")
    exit()

# Initial joint positions
joint_positions = {
    "J1": 0.400,
    "J2": -101.980,
    "J3": 162.100,
    "J4": 0.560,
    "J5": 43.930,
    "J6": 1.730,
}

# Webcam
cap = cv2.VideoCapture(0)
cap.set(3, 1280)  # Set width of the webcam frame
cap.set(4, 720)  # Set height of the webcam frame

# Hand Detector
detector = HandDetector(detectionCon=0.8, maxHands=1)

# Calibration for distance conversion
pixels_per_cm = 37

def send_message_and_confirm(sock, message):
    try:
        print(f"Sending message: {message}")
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()  # Assuming the server sends a response
        print(f"Received response: {response}")
        return response.strip() == "True"  # Assuming the server sends "True" on success
    except Exception as e:
        print(f"Error sending/receiving message: {e}")
        return False

# Initialize variables for finger tracking
initial_position_x = None
initial_position_y = None
fingers_count = None
joint_key = None

while True:
    success, img = cap.read()
    if not success:
        print("Error: Failed to capture image from webcam.")
        break
    img = cv2.flip(img, 1)  # Flip the image horizontally

    # Detect hands and fingers
    hands, img = detector.findHands(img, draw=True)

    # Display finger count
    if fingers_count is not None:
        cv2.putText(img, f'Fingers Count: {fingers_count}', (50, 150),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    if hands:
        hand = hands[0]
        lmList = hand["lmList"]  # List of landmarks

        # Calculate thumb and index finger distances
        index_finger_tip = lmList[8]
        thumb_tip = lmList[4]
        thumb_index_distance = ((thumb_tip[0] - index_finger_tip[0]) ** 2 + (
                    thumb_tip[1] - index_finger_tip[1]) ** 2) ** 0.5

        if thumb_index_distance < 20:  # Assuming distance of 20 pixels as close enough
            if initial_position_x is None:
                initial_position_x = lmList[8][0]  # Set the initial position for x movement
            if initial_position_y is None:
                initial_position_y = lmList[8][1]  # Set the initial position for y movement

            current_position_x = lmList[8][0]
            dx = current_position_x - initial_position_x  # Calculate the difference in x-coordinates
            distance_cm_x = dx / pixels_per_cm  # Convert pixels to centimeters for x movement

            current_position_y = lmList[8][1]
            dy = current_position_y - initial_position_y  # Calculate the difference in y-coordinates
            distance_cm_y = dy / pixels_per_cm  # Convert pixels to centimeters for y movement

            cvzone.putTextRect(img, f'Moved X: {distance_cm_x:.2f} cm', (50, 50))
            cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 100))

            joint_value = None

            # Determine finger count
            fingers = detector.fingersUp(hand)
            fingers_count = fingers.count(1)

            # Display finger count
            cv2.putText(img, f'Fingers Count: {fingers_count}', (50, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

            # Determine which joint to control based on finger count
            joint_keys = ["J1", "J2", "J3", "J4", "J5"]
            if 1 <= fingers_count <= 5:
                joint_key = joint_keys[fingers_count - 1]
                joint_value = joint_positions[joint_key]

                if fingers_count in [1, 4]:  # J1 and J4 are controlled by x-movement
                    movement = distance_cm_x
                elif fingers_count in [2, 5]:  # J2 and J5 are controlled by y-movement
                    movement = distance_cm_y
                elif fingers_count == 3:  # J3 is controlled by inverted y-movement
                    movement = -distance_cm_y

                # Perform joint manipulation
                if joint_value is not None:
                    # Determine if the movement is positive or negative
                    if movement > 0:
                        adjustment = 5 if int(movement) % 2 == 0 else 0
                    else:
                        adjustment = -5 if int(abs(movement)) % 2 == 0 else 0

                    joint_value += adjustment
                    joint_positions[joint_key] = joint_value

                    message = f"1,({joint_positions['J1']:.3f},{joint_positions['J2']:.3f},{joint_positions['J3']:.3f},{joint_positions['J4']:.3f},{joint_positions['J5']:.3f},{joint_positions['J6']:.3f})(3,0)"
                    send_message_and_confirm(sock, message)

    # Display manipulating joint information
    if joint_key is not None:
        cv2.putText(img, f'Manipulating Joint: {joint_key}', (50, 200),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
sock.close()



Sending message: 1,(-4.600,-101.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-96.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-101.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-106.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-101.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-96.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-91.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-86.980,162.100,0.560,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-86.980,162.100,-4.440,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-86.980,162.100,-9.440,43.930,1.730)(3,0)
Received response: True
Sending message: 1,(-4.600,-86.980,162.100,-14.440,43.930,1.730)(3,0)
Rec

In [4]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import cvzone
import socket

HOST = "192.168.1.20"
PORT = 10003

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    sock.connect((HOST, PORT))
except ConnectionRefusedError:
    print("Error: Connection refused. Please check if the server is running.")
    exit()

J1 = 0.400
J2 = -113.980
J3 = 162.100
J4 = 0.560
J5 = 43.930
J6 = 1.730

cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)

detector = HandDetector(detectionCon=0.8, maxHands=1)

pixels_per_cm = 37

initial_position_x = None
initial_position_y = None
override = False

def send_message_and_confirm(sock, message):
    try:
        print(f"Sending message: {message}")
        sock.sendall(message.encode())
        response = sock.recv(1024).decode()
        print(f"Received response: {response}")
        return response.strip() == "True"
    except Exception as e:
        print(f"Error sending message: {e}")
        return False

def horizontal_change(lmList, initial_position_x):
    current_position_x = lmList[8][0]
    dx = current_position_x - initial_position_x 
    distance_cm_x = dx / pixels_per_cm
    return distance_cm_x, current_position_x

def vertical_change(lmList, initial_position_y):
    current_position_y = lmList[8][1]
    dy = current_position_y - initial_position_y
    distance_cm_y = dy / pixels_per_cm
    return distance_cm_y, current_position_y

def changes(joint_value, movement, Joint_num):
    if Joint_num in ['J1', 'J3', 'J4', 'J5']:
        if movement > 0:                
            adjustment = 5 if int(movement) % 2 == 0 else 0
        else:
            adjustment = -5 if int(abs(movement)) % 2 == 0 else 0
        
        new_joint_value = joint_value + adjustment
    elif Joint_num == 'J2':
        if movement < 0:                
            adjustment = 3 if int(movement) % 2 == 0 else 0
        else:
            adjustment = -3 if int(abs(movement)) % 2 == 0 else 0
        
        new_joint_value = joint_value + adjustment
    
    return new_joint_value


def is_thumb_and_pinky_joined(lmList):
    thumb_tip = 4
    pinky_tip = 20

    thumb_x, thumb_y = lmList[thumb_tip][1], lmList[thumb_tip][2]
    pinky_x, pinky_y = lmList[pinky_tip][1], lmList[pinky_tip][2]
    distance = ((thumb_x - pinky_x) ** 2 + (thumb_y - pinky_y) ** 2) ** 0.5

    threshold = 30 
    return distance < threshold

def is_thumb_and_ring_joined(lmList):
    thumb_tip = 4
    ring_tip = 16

    thumb_x, thumb_y = lmList[thumb_tip][1], lmList[thumb_tip][2]
    ring_x, ring_y = lmList[ring_tip][1], lmList[ring_tip][2]
    distance = ((thumb_x - ring_x) ** 2 + (thumb_y - ring_y) ** 2) ** 0.5

    threshold = 30 
    return distance < threshold

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)

    hands, img = detector.findHands(img, draw=True)
    updated = False
    joint_finger_detected = False

    if hands:
        hand = hands[0]
        lmList = hand["lmList"]
        bbox = hand["bbox"]
        fingers = detector.fingersUp(hand)

        fingers_count = fingers.count(1)
        
        if fingers_count > 0:
            joint_finger_detected = True
        
        if joint_finger_detected:

            #J1 accessing
            if fingers == [1, 1,1, 0, 0]:   
                if initial_position_x is None:
                    initial_position_x = lmList[8][0]
                
                distance_cm_x, current_position_x = horizontal_change(lmList, initial_position_x)
                cvzone.putTextRect(img, f'Moved X: {distance_cm_x:.2f} cm', (50, 50))
                cvzone.putTextRect(img, "J1 activated", (50, 100)) 
                J1 = changes(J1, distance_cm_x, 'J1')

                if J1 > 239.000:
                    J1 = 239.000
                elif J1 < -239.930:
                    J1 = -239.000

                message = f"1,({J1:.3f},{J2:.3f},{J3:.3f},{J4:.3f},{J5:.3f},{J6:.3f})(3,0)"
                send = send_message_and_confirm(sock, message)

                if send:
                    updated = True
                    print(f'Coordinates passed {message} for J1')

                if updated:
                    initial_position_x = current_position_x

                if fingers == [1, 0, 0, 0, 1]:  
                    initial_position_x = None                

            #J2 accessing
            if fingers == [1, 0, 0, 0, 0]:  
                if initial_position_y is None:
                    initial_position_y = lmList[8][1]

                distance_cm_y, current_position_y = vertical_change(lmList, initial_position_y)
                cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 50))
                cvzone.putTextRect(img, "J2 activated", (50, 100)) 
                J2 = changes(J2, distance_cm_y, 'J2')

                if J2 < -113.990:
                    J2 = -113.000
                elif  J2 > 85 :
                    J2 = 85

                message = f"1,({J1:.3f},{J2:.3f},{J3:.3f},{J4:.3f},{J5:.3f},{J6:.3f})(3,0)"
                send = send_message_and_confirm(sock, message)

                if send:
                    updated = True
                    print(f'Coordinates passed {message} for J2')

                if updated:
                    initial_position_y = current_position_y

                if fingers == [1, 1, 0, 0, 1]:
                    initial_position_y = None

            #J3 accessing
            if fingers == [1, 1, 0, 0, 0]:
                if initial_position_y is None:
                    initial_position_y = lmList[8][1]

                distance_cm_y, current_position_y = vertical_change(lmList, initial_position_y)
                cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 50))
                cvzone.putTextRect(img, "J3 activated", (50, 100))
                J3 = changes(J3, distance_cm_y, 'J3')

                if J3 > 161.420:
                    J3 = 161.420
                elif J3 < 5:
                    J3= 5

                message = f"1,({J1:.3f},{J2:.3f},{J3:.3f},{J4:.3f},{J5:.3f},{J6:.3f})(3,0)"
                send = send_message_and_confirm(sock, message)

                if send:
                    updated = True
                    print(f'Coordinates passed {message} for J3')

                if updated:
                    initial_position_y = current_position_y

                if fingers == [1, 1, 1, 0, 1]:
                    initial_position_y = None

            #J4 accessing
            '''if fingers == [1, 1, 1, 1, 0]:
                if initial_position_x is None:
                    initial_position_x = lmList[8][0]

                distance_cm_x, current_position_x = horizontal_change(lmList, initial_position_x)
                cvzone.putTextRect(img, f'Moved X: {distance_cm_x:.2f} cm', (50, 50))
                cvzone.putTextRect(img, "J4 activated", (50, 100))
                J4 = changes(J4, distance_cm_x, 'J4')

                if J4  > 199:
                    J4 = 199
                elif J4 < -199:
                    J4 = -199

                message = f"1,({J1:.3f},{J2:.3f},{J3:.3f},{J4:.3f},{J5:.3f},{J6:.3f})(3,0)"
                send = send_message_and_confirm(sock, message)

                if send:
                    updated = True
                    print(f'Coordinates passed {message} for J4')

                if updated:
                    initial_position_x = current_position_x

                if fingers == [1, 1, 1, 0, 1]:
                    initial_position_x = None'''

            #J5 accessing
            if fingers == [0, 1, 1, 1, 1]:
                if initial_position_y is None:
                    initial_position_y = lmList[8][1]
                
                distance_cm_y, current_position_y = vertical_change(lmList, initial_position_y)
                cvzone.putTextRect(img, f'Moved Y: {distance_cm_y:.2f} cm', (50, 50))
                cvzone.putTextRect(img, "J5 activated", (50, 100))
                J5 = changes(J5, distance_cm_y, 'J5')

                if J5  > 119.970:
                    J5 = 119.000
                elif J5 < -95:
                    J5 = -90

                message = f"1,({J1:.3f},{J2:.3f},{J3:.3f},{J4:.3f},{J5:.3f},{J6:.3f})(3,0)"
                send = send_message_and_confirm(sock, message)

                if send:
                    updated = True
                    print(f'Coordinates passed {message} for J5')

                if updated:
                    initial_position_y = current_position_y

                if fingers == [1, 1, 1, 0, 1]:
                    initial_position_y = None
            
            #Gripper Opened
            if fingers==[1,1,1,1,1]:
                cvzone.putTextRect(img, "Gripper Opened", (50, 100))
                message = f"2"
                send  = send_message_and_confirm(sock,message)

                if send:
                    print("Gripper Opened")

            #Gripper Closed
            if fingers == [0,0,0,1,1]:
                cvzone.putTextRect(img,"Gripper Closed",(50,100))
                message = f"3"
                send = send_message_and_confirm(sock,message)

                if send:
                    print("Gripper Closed")

            #Override making Slow
            if fingers == [ 0,1,1,0,0]:   #(index, middle)
                cvzone.putTextRect(img,"Speed Decreased",(50,100))
                message = f"4"
                send = send_message_and_confirm(sock,message)

                if send:
                    print('Speed Decreased')   

            if fingers == [0,1,1,1,0]:
                cvzone.putTextRect(img,"Speed Increased",(50,100))
                message = "5"
                send = send_message_and_confirm(sock,message)

                if send:
                    print("Speed Increased")                                  
            
                     

    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
sock.close()


Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending message: 2
Received response: True
Gripper Opened
Sending messag

KeyboardInterrupt: 